In [1]:
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from konlpy.tag import Mecab
from models import AE
from scipy.stats import ortho_group, wasserstein_distance

# from googletrans import Translator
from models.transformer import * 

import time
import numpy as np

import os
import sys
import urllib.request
import requests
import datetime
import shutil
import pickle
import json

In [2]:
with open('./data/ko_noun_dict(vocab_size-1250).pkl', "rb") as f:
    ko_dict = pickle.load(f)
    
with open('./data/en_noun_dict(vocab_size-1250).pkl', "rb") as f:
    en_dict = pickle.load(f)

In [3]:
with open('./data/ko_noun_dict(vocab_size-1250).pkl', "rb") as f:
    ko_noun_dict = pickle.load(f)
    
with open('./data/en_noun_dict(vocab_size-1250).pkl', "rb") as f:
    en_noun_dict = pickle.load(f)
    
with open('./data/ko_verb_dict(vocab_size-547).pkl', "rb") as f:
    ko_verb_dict = pickle.load(f)
    
with open('./data/en_verb_dict(vocab_size-547).pkl', "rb") as f:
    en_verb_dict = pickle.load(f)
    
with open('./data/ko_v_1.pkl', "rb") as f:
    ko_adjective_dict = pickle.load(f)
    
with open('./data/en_v_1.pkl', "rb") as f:
    en_adjective_dict = pickle.load(f)
    
with open('./data/ko_verb_1.pkl', "rb") as f:
    ko_verb_dict_2 = pickle.load(f)
    
with open('./data/en_verb_1.pkl', "rb") as f:
    en_verb_dict_2 = pickle.load(f)

In [4]:
ko_data = np.array(list(ko_noun_dict.values()) + list(ko_verb_dict.values()) + list(ko_verb_dict_2.values()) + list(ko_adjective_dict.values()))
en_data = np.array(list(en_noun_dict.values()) + list(en_verb_dict.values()) + list(en_verb_dict_2.values()) + list(en_adjective_dict.values())) 
ko_noun_data = np.array(list(ko_noun_dict.values()))
en_noun_data = np.array(list(en_noun_dict.values())) 
ko_verb_data = np.array(list(ko_verb_dict.values()) + list(ko_verb_dict_2.values()))
en_verb_data = np.array(list(en_verb_dict.values()) + list(en_verb_dict_2.values())) 
ko_adjective_data = np.array(list(ko_adjective_dict.values()))
en_adjective_data = np.array(list(en_adjective_dict.values()))

In [5]:
latent_dim = 250
autoencoder = AE.AutoEncoder(latent_dim)

In [6]:
def autoencoder_test(latent_dim, train_data, label_data, ko_data, en_data, num_sample=500, shuffle=True):
    
    autoencoder = AE.AutoEncoder(latent_dim)
    autoencoder.compile(optimizer='adam', loss=tf.losses.MeanSquaredError())
    autoencoder.fit(train_data, label_data, epochs=100, shuffle=True)
    
    wasserstein_error = 0
    for i in range(len(ko_data)):

        wasserstein_error += wasserstein_distance(ko_data[i], autoencoder(en_data)[i])

    print(wasserstein_error / len(ko_data))

In [12]:
def linear_test(latent_dim, train_data, label_data, ko_data, en_data, num_sample=500, shuffle=True):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(300, bias_initializer='zero'))
    opt = tf.keras.optimizers.Adam()
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50)
    model.fit(en_noun_data, ko_noun_data, epochs=500, batch_size=64, callbacks=[es]) 
    
    wasserstein_error = 0
    for i in range(len(ko_data)):

        wasserstein_error += wasserstein_distance(ko_data[i], model(en_data)[i])

    print(wasserstein_error / len(ko_data))

In [8]:
shuffle_idx = np.arange((ko_data.shape[0]))
np.random.shuffle(shuffle_idx)

In [9]:
shuffle_idx

array([2310,  789, 1338, ...,  573, 3010,  243])

In [10]:
autoencoder_test(250, ko_data[shuffle_idx][:500], en_data[shuffle_idx][:500], ko_data, en_data)

Train on 500 samples
Epoch 1/100
500/500 [==============================] - 0s 491us/sample - loss: 0.2257
Epoch 2/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0731
Epoch 3/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0602
Epoch 4/100
500/500 [==============================] - 0s 55us/sample - loss: 0.0591
Epoch 5/100
500/500 [==============================] - 0s 57us/sample - loss: 0.0583
Epoch 6/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0578
Epoch 7/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0574
Epoch 8/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0570
Epoch 9/100
500/500 [==============================] - 0s 57us/sample - loss: 0.0566
Epoch 10/100
500/500 [==============================] - 0s 57us/sample - loss: 0.0563
Epoch 11/100
500/500 [==============================] - 0s 56us/sample - loss: 0.0559
Epoch 12/100
500/500 [===================

500/500 [==============================] - 0s 54us/sample - loss: 0.0445
Epoch 97/100
500/500 [==============================] - 0s 54us/sample - loss: 0.0444
Epoch 98/100
500/500 [==============================] - 0s 55us/sample - loss: 0.0444
Epoch 99/100
500/500 [==============================] - 0s 55us/sample - loss: 0.0443
Epoch 100/100
500/500 [==============================] - 0s 54us/sample - loss: 0.0442
0.26684758558800037


In [13]:
linear_test(250, ko_data[shuffle_idx][:500], en_data[shuffle_idx][:500], ko_data, en_data)

Train on 1250 samples
Epoch 1/500
1250/1250 [==============================] - 0s 206us/sample - loss: 0.1319 - accuracy: 0.0472
Epoch 2/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0947 - accuracy: 0.1024
Epoch 3/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0800 - accuracy: 0.1184
Epoch 4/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0709 - accuracy: 0.1392
Epoch 5/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0644 - accuracy: 0.1648
Epoch 6/500
1250/1250 [==============================] - 0s 32us/sample - loss: 0.0595 - accuracy: 0.1840
Epoch 7/500
1250/1250 [==============================] - 0s 33us/sample - loss: 0.0556 - accuracy: 0.2032
Epoch 8/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0525 - accuracy: 0.2096
Epoch 9/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0499 - accuracy: 0.2200
Epoch 10/500
1250/1250 

1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3336
Epoch 78/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3328
Epoch 79/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3344
Epoch 80/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3280
Epoch 81/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3336
Epoch 82/500
1250/1250 [==============================] - 0s 28us/sample - loss: 0.0334 - accuracy: 0.3320
Epoch 83/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3320
Epoch 84/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3328
Epoch 85/500
1250/1250 [==============================] - 0s 28us/sample - loss: 0.0333 - accuracy: 0.3312
Epoch 86/500
1250/1250 [==========================

1250/1250 [==============================] - 0s 31us/sample - loss: 0.0332 - accuracy: 0.3304
Epoch 154/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0333 - accuracy: 0.3264
Epoch 155/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0332 - accuracy: 0.3336
Epoch 156/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0332 - accuracy: 0.3312
Epoch 157/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0333 - accuracy: 0.3248
Epoch 158/500
1250/1250 [==============================] - 0s 30us/sample - loss: 0.0333 - accuracy: 0.3352
Epoch 159/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0332 - accuracy: 0.3304
Epoch 160/500
1250/1250 [==============================] - 0s 31us/sample - loss: 0.0333 - accuracy: 0.3344
Epoch 161/500
1250/1250 [==============================] - 0s 30us/sample - loss: 0.0332 - accuracy: 0.3320
Epoch 162/500
1250/1250 [=================

Epoch 229/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3272
Epoch 230/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3296
Epoch 231/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3312
Epoch 232/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3352
Epoch 233/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3272
Epoch 234/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0332 - accuracy: 0.3320
Epoch 235/500
1250/1250 [==============================] - 0s 29us/sample - loss: 0.0333 - accuracy: 0.3336
Epoch 236/500
1250/1250 [==============================] - 0s 28us/sample - loss: 0.0333 - accuracy: 0.3336
Epoch 237/500
1250/1250 [==============================] - 0s 28us/sample - loss: 0.0333 - accuracy: 0.3296
Epoch 238/500
1250/1250 [===